<a href="https://www.kaggle.com/code/vtrackstar/most-statistically-similar-nba-seasons?scriptVersionId=201111035" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nba-aba-baa-stats/Advanced.csv
/kaggle/input/nba-aba-baa-stats/Player Season Info.csv
/kaggle/input/nba-aba-baa-stats/Player Career Info.csv
/kaggle/input/nba-aba-baa-stats/Player Shooting.csv
/kaggle/input/nba-aba-baa-stats/Opponent Stats Per Game.csv
/kaggle/input/nba-aba-baa-stats/Player Totals.csv
/kaggle/input/nba-aba-baa-stats/Opponent Stats Per 100 Poss.csv
/kaggle/input/nba-aba-baa-stats/Per 100 Poss.csv
/kaggle/input/nba-aba-baa-stats/Team Summaries.csv
/kaggle/input/nba-aba-baa-stats/Player Per Game.csv
/kaggle/input/nba-aba-baa-stats/All-Star Selections.csv
/kaggle/input/nba-aba-baa-stats/End of Season Teams (Voting).csv
/kaggle/input/nba-aba-baa-stats/Player Play By Play.csv
/kaggle/input/nba-aba-baa-stats/End of Season Teams.csv
/kaggle/input/nba-aba-baa-stats/Team Stats Per 100 Poss.csv
/kaggle/input/nba-aba-baa-stats/Per 36 Minutes.csv
/kaggle/input/nba-aba-baa-stats/Opponent Totals.csv
/kaggle/input/nba-aba-baa-stats/Team Abbrev.csv
/kaggle/input/nba-aba-b

In [2]:
playerpg = pd.read_csv('/kaggle/input/nba-aba-baa-stats/Player Per Game.csv')
playerpbp = pd.read_csv('/kaggle/input/nba-aba-baa-stats/Player Play By Play.csv')
playeraw = pd.read_csv('/kaggle/input/nba-aba-baa-stats/Player Award Shares.csv')
playersh = pd.read_csv('/kaggle/input/nba-aba-baa-stats/Player Shooting.csv')

print(playerpg.columns)
print(playerpbp.columns)
print(playeraw.columns)
print(playersh.columns)

Index(['seas_id', 'season', 'player_id', 'player', 'birth_year', 'pos', 'age',
       'experience', 'lg', 'tm', 'g', 'gs', 'mp_per_game', 'fg_per_game',
       'fga_per_game', 'fg_percent', 'x3p_per_game', 'x3pa_per_game',
       'x3p_percent', 'x2p_per_game', 'x2pa_per_game', 'x2p_percent',
       'e_fg_percent', 'ft_per_game', 'fta_per_game', 'ft_percent',
       'orb_per_game', 'drb_per_game', 'trb_per_game', 'ast_per_game',
       'stl_per_game', 'blk_per_game', 'tov_per_game', 'pf_per_game',
       'pts_per_game'],
      dtype='object')
Index(['seas_id', 'season', 'player_id', 'player', 'birth_year', 'pos', 'age',
       'experience', 'lg', 'tm', 'g', 'mp', 'pg_percent', 'sg_percent',
       'sf_percent', 'pf_percent', 'c_percent',
       'on_court_plus_minus_per_100_poss', 'net_plus_minus_per_100_poss',
       'bad_pass_turnover', 'lost_ball_turnover', 'shooting_foul_committed',
       'offensive_foul_committed', 'shooting_foul_drawn',
       'offensive_foul_drawn', 'points_gener

In [3]:
con_perg = playerpg[['seas_id', 'season', 'player', 'tm', 'experience', 'age', 'pos','g', 'mp_per_game', 'fg_per_game',
       'fga_per_game', 'fg_percent', 'x3p_per_game', 'x3pa_per_game',
       'x3p_percent', 'x2p_per_game', 'x2pa_per_game', 'x2p_percent',
       'e_fg_percent', 'ft_per_game', 'fta_per_game', 'ft_percent',
       'orb_per_game', 'drb_per_game', 'trb_per_game', 'ast_per_game',
       'stl_per_game', 'blk_per_game', 'tov_per_game', 'pf_per_game',
       'pts_per_game']].copy()
con_pbp = playerpbp[['seas_id', 'pg_percent', 'sg_percent',
       'sf_percent', 'pf_percent', 'c_percent',
       'on_court_plus_minus_per_100_poss', 'net_plus_minus_per_100_poss',
       'bad_pass_turnover', 'lost_ball_turnover', 'shooting_foul_committed',
       'offensive_foul_committed', 'shooting_foul_drawn',
       'offensive_foul_drawn', 'points_generated_by_assists', 'and1',
       'fga_blocked']].copy()
con_award = playeraw[['seas_id', 'first', 'pts_won', 'pts_max',
       'share', 'winner']].copy()
con_shot = playersh[['seas_id', 'avg_dist_fga',
       'percent_fga_from_x2p_range', 'percent_fga_from_x0_3_range',
       'percent_fga_from_x3_10_range', 'percent_fga_from_x10_16_range',
       'percent_fga_from_x16_3p_range', 'percent_fga_from_x3p_range',
       'fg_percent_from_x2p_range', 'fg_percent_from_x0_3_range',
       'fg_percent_from_x3_10_range', 'fg_percent_from_x10_16_range',
       'fg_percent_from_x16_3p_range', 'fg_percent_from_x3p_range',
       'percent_assisted_x2p_fg', 'percent_assisted_x3p_fg',
       'percent_dunks_of_fga', 'num_of_dunks', 'percent_corner_3s_of_3pa',
       'corner_3_point_percent']].copy()

merged_player = con_perg.merge(con_pbp, on='seas_id', how='outer') \
               .merge(con_shot, on='seas_id', how='outer')
print(merged_player.head())
print(merged_player.columns)

   seas_id  season         player   tm  experience   age  pos   g  \
0        1    1947   Al Brightman  BOS           1  23.0    F  58   
1        2    1947      Al Lujack  WSC           1  26.0    F   5   
2        3    1947    Al Negratti  WSC           1  25.0  F-C  11   
3        4    1947    Angelo Musi  PHW           1  28.0    G  60   
4        5    1947  Ariel Maughan  DTF           1  23.0    F  59   

   mp_per_game  fg_per_game  ...  fg_percent_from_x3_10_range  \
0          NaN          3.8  ...                          NaN   
1          NaN          0.2  ...                          NaN   
2          NaN          1.2  ...                          NaN   
3          NaN          3.8  ...                          NaN   
4          NaN          3.8  ...                          NaN   

   fg_percent_from_x10_16_range  fg_percent_from_x16_3p_range  \
0                           NaN                           NaN   
1                           NaN                           NaN   

In [4]:
preshai = merged_player[merged_player['player'] == 'Shai Gilgeous-Alexander']  # For string value
preshai.head()

,seas_id,season,player,tm,experience,age,pos,g,mp_per_game,fg_per_game,...,fg_percent_from_x3_10_range,fg_percent_from_x10_16_range,fg_percent_from_x16_3p_range,fg_percent_from_x3p_range,percent_assisted_x2p_fg,percent_assisted_x3p_fg,percent_dunks_of_fga,num_of_dunks,percent_corner_3s_of_3pa,corner_3_point_percent
28162,28163,2019,Shai Gilgeous-Alexander,LAC,1,20.0,PG,82,26.5,4.2,...,0.417,0.415,0.436,0.367,0.310,0.902,0.028,19.0,0.460,0.406
28838,28839,2020,Shai Gilgeous-Alexander,OKC,2,21.0,SG,70,34.7,6.8,...,0.404,0.500,0.507,0.347,0.235,0.540,0.019,19.0,0.127,0.438
29540,29541,2021,Shai Gilgeous-Alexander,OKC,3,22.0,SG,35,33.7,8.2,...,0.528,0.382,0.360,0.418,0.074,0.296,0.018,10.0,0.076,0.462
30336,30337,2022,Shai Gilgeous-Alexander,OKC,4,23.0,PG,56,34.7,8.5,...,0.445,0.442,0.490,0.300,0.137,0.303,0.023,23.0,0.077,0.391
31045,31046,2023,Shai Gilgeous-Alexander,OKC,5,24.0,PG,68,35.5,10.4,...,0.438,0.512,0.396,0.345,0.201,0.310,0.039,50.0,0.101,0.235


In [5]:
# Count the number of NaN values in each column
nan_counts = merged_player.isna().sum()

# Display columns with NaN values
nan_columns = nan_counts[nan_counts > 0]

# Print the columns and their respective NaN counts
print(nan_columns)

age                                    22
mp_per_game                          1083
fg_percent                            157
x3p_per_game                         6352
x3pa_per_game                        6352
x3p_percent                         10528
x2p_percent                           239
e_fg_percent                          157
ft_percent                           1291
orb_per_game                         4657
drb_per_game                         4657
trb_per_game                          894
stl_per_game                         5626
blk_per_game                         5625
tov_per_game                         5635
pg_percent                          26043
sg_percent                          23992
sf_percent                          23605
pf_percent                          23143
c_percent                           24970
on_court_plus_minus_per_100_poss    15094
net_plus_minus_per_100_poss         15094
bad_pass_turnover                   15085
lost_ball_turnover                

In [6]:
similarity_columns = [
'fg_per_game', 'fga_per_game',
       'x3p_per_game', 'x3pa_per_game', 'x3p_percent', 'x2p_per_game',
       'x2pa_per_game', 'x2p_percent', 'ft_per_game',
       'fta_per_game', 'ft_percent', 'orb_per_game', 'drb_per_game',
       'trb_per_game', 'ast_per_game', 'stl_per_game', 'blk_per_game',
       'tov_per_game', 'pf_per_game', 'pts_per_game',
       'on_court_plus_minus_per_100_poss', 'net_plus_minus_per_100_poss',
       'bad_pass_turnover', 'lost_ball_turnover', 'shooting_foul_committed',
       'offensive_foul_committed', 'shooting_foul_drawn',
       'offensive_foul_drawn', 'points_generated_by_assists', 'and1',
       'fga_blocked', 'percent_assisted_x2p_fg',
       'percent_assisted_x3p_fg', 'percent_dunks_of_fga',
       'percent_corner_3s_of_3pa', 'corner_3_point_percent'
]

In [7]:
shai = merged_player[merged_player['player'] == 'Shai Gilgeous-Alexander']
pd.set_option('display.max_columns', None)  # Show all columns

shai.head()

,seas_id,season,player,tm,experience,age,pos,g,mp_per_game,fg_per_game,fga_per_game,fg_percent,x3p_per_game,x3pa_per_game,x3p_percent,x2p_per_game,x2pa_per_game,x2p_percent,e_fg_percent,ft_per_game,fta_per_game,ft_percent,orb_per_game,drb_per_game,trb_per_game,ast_per_game,stl_per_game,blk_per_game,tov_per_game,pf_per_game,pts_per_game,pg_percent,sg_percent,sf_percent,pf_percent,c_percent,on_court_plus_minus_per_100_poss,net_plus_minus_per_100_poss,bad_pass_turnover,lost_ball_turnover,shooting_foul_committed,offensive_foul_committed,shooting_foul_drawn,offensive_foul_drawn,points_generated_by_assists,and1,fga_blocked,avg_dist_fga,percent_fga_from_x2p_range,percent_fga_from_x0_3_range,percent_fga_from_x3_10_range,percent_fga_from_x10_16_range,percent_fga_from_x16_3p_range,percent_fga_from_x3p_range,fg_percent_from_x2p_range,fg_percent_from_x0_3_range,fg_percent_from_x3_10_range,fg_percent_from_x10_16_range,fg_percent_from_x16_3p_range,fg_percent_from_x3p_range,percent_assisted_x2p_fg,percent_assisted_x3p_fg,percent_dunks_of_fga,num_of_dunks,percent_corner_3s_of_3pa,corner_3_point_percent
28162,28163,2019,Shai Gilgeous-Alexander,LAC,1,20.0,PG,82,26.5,4.2,8.7,0.476,0.6,1.7,0.367,3.5,7.0,0.503,0.512,1.9,2.4,0.800,0.7,2.1,2.8,3.3,1.2,0.5,1.7,2.1,10.8,50.0,49.0,2.0,NaN,NaN,-1.9,-6.4,82.0,26.0,81.0,16.0,85.0,9.0,654.0,21.0,67.0,11.2,0.806,0.314,0.161,0.222,0.109,0.194,0.503,0.631,0.417,0.415,0.436,0.367,0.310,0.902,0.028,19.0,0.460,0.406
28838,28839,2020,Shai Gilgeous-Alexander,OKC,2,21.0,SG,70,34.7,6.8,14.5,0.471,1.2,3.6,0.347,5.6,10.9,0.512,0.514,4.1,5.1,0.807,0.7,5.2,5.9,3.3,1.1,0.7,1.9,1.7,19.0,6.0,78.0,17.0,NaN,NaN,2.7,3.4,61.0,40.0,53.0,15.0,148.0,9.0,561.0,35.0,62.0,11.4,0.753,0.286,0.263,0.136,0.068,0.247,0.512,0.617,0.404,0.500,0.507,0.347,0.235,0.540,0.019,19.0,0.127,0.438
29540,29541,2021,Shai Gilgeous-Alexander,OKC,3,22.0,SG,35,33.7,8.2,16.1,0.508,2.0,4.9,0.418,6.2,11.3,0.547,0.571,5.3,6.5,0.808,0.5,4.2,4.7,5.9,0.8,0.7,3.0,2.0,23.7,7.0,31.0,62.0,1.0,NaN,-7.9,4.4,59.0,31.0,29.0,7.0,87.0,4.0,523.0,21.0,40.0,11.6,0.699,0.333,0.225,0.097,0.044,0.301,0.547,0.633,0.528,0.382,0.360,0.418,0.074,0.296,0.018,10.0,0.076,0.462
30336,30337,2022,Shai Gilgeous-Alexander,OKC,4,23.0,PG,56,34.7,8.5,18.8,0.453,1.6,5.3,0.300,6.9,13.5,0.514,0.496,5.9,7.2,0.810,0.7,4.3,5.0,5.9,1.3,0.8,2.8,2.5,24.5,65.0,33.0,2.0,NaN,NaN,-6.8,2.9,78.0,50.0,61.0,19.0,168.0,6.0,806.0,35.0,64.0,12.1,0.718,0.220,0.320,0.131,0.047,0.282,0.514,0.662,0.445,0.442,0.490,0.300,0.137,0.303,0.023,23.0,0.077,0.391
31045,31046,2023,Shai Gilgeous-Alexander,OKC,5,24.0,PG,68,35.5,10.4,20.3,0.510,0.9,2.5,0.345,9.5,17.8,0.533,0.531,9.8,10.9,0.905,0.9,4.0,4.8,5.5,1.6,1.0,2.8,2.8,31.4,84.0,17.0,NaN,NaN,NaN,3.1,4.9,84.0,64.0,78.0,30.0,322.0,15.0,942.0,57.0,97.0,9.8,0.878,0.283,0.276,0.246,0.073,0.122,0.533,0.678,0.438,0.512,0.396,0.345,0.201,0.310,0.039,50.0,0.101,0.235


In [8]:
brunson = merged_player[merged_player['player'] == 'Jalen Brunson']
brunson.head()

,seas_id,season,player,tm,experience,age,pos,g,mp_per_game,fg_per_game,fga_per_game,fg_percent,x3p_per_game,x3pa_per_game,x3p_percent,x2p_per_game,x2pa_per_game,x2p_percent,e_fg_percent,ft_per_game,fta_per_game,ft_percent,orb_per_game,drb_per_game,trb_per_game,ast_per_game,stl_per_game,blk_per_game,tov_per_game,pf_per_game,pts_per_game,pg_percent,sg_percent,sf_percent,pf_percent,c_percent,on_court_plus_minus_per_100_poss,net_plus_minus_per_100_poss,bad_pass_turnover,lost_ball_turnover,shooting_foul_committed,offensive_foul_committed,shooting_foul_drawn,offensive_foul_drawn,points_generated_by_assists,and1,fga_blocked,avg_dist_fga,percent_fga_from_x2p_range,percent_fga_from_x0_3_range,percent_fga_from_x3_10_range,percent_fga_from_x10_16_range,percent_fga_from_x16_3p_range,percent_fga_from_x3p_range,fg_percent_from_x2p_range,fg_percent_from_x0_3_range,fg_percent_from_x3_10_range,fg_percent_from_x10_16_range,fg_percent_from_x16_3p_range,fg_percent_from_x3p_range,percent_assisted_x2p_fg,percent_assisted_x3p_fg,percent_dunks_of_fga,num_of_dunks,percent_corner_3s_of_3pa,corner_3_point_percent
27863,27864,2019,Jalen Brunson,DAL,1,22.0,PG,73,21.8,3.6,7.7,0.467,0.9,2.5,0.348,2.8,5.3,0.523,0.523,1.2,1.6,0.725,0.3,2.0,2.3,3.2,0.5,0.1,1.2,1.7,9.3,75.0,23.0,1.0,NaN,NaN,-3.2,-2.4,51.0,21.0,53.0,5.0,49.0,40.0,555.0,11.0,43.0,13.3,0.680,0.218,0.225,0.161,0.076,0.320,0.523,0.707,0.441,0.451,0.395,0.348,0.338,0.762,0.009,4.0,0.293,0.302
28550,28551,2020,Jalen Brunson,DAL,2,23.0,PG,57,17.9,3.2,6.8,0.466,0.8,2.1,0.358,2.4,4.7,0.515,0.522,1.1,1.3,0.813,0.4,2.0,2.4,3.3,0.4,0.1,1.2,1.3,8.2,96.0,4.0,NaN,NaN,NaN,8.1,4.5,33.0,16.0,32.0,9.0,30.0,26.0,477.0,12.0,11.0,13.6,0.691,0.178,0.258,0.139,0.116,0.309,0.515,0.623,0.500,0.500,0.400,0.358,0.239,0.814,0.000,0.0,0.233,0.357
29237,29238,2021,Jalen Brunson,DAL,3,24.0,PG,68,25.0,4.8,9.2,0.523,1.2,2.9,0.405,3.6,6.3,0.578,0.588,1.8,2.2,0.795,0.4,3.0,3.4,3.5,0.5,0.0,1.2,1.6,12.6,78.0,22.0,NaN,NaN,NaN,4.4,4.2,53.0,13.0,37.0,10.0,59.0,34.0,602.0,15.0,17.0,13.3,0.681,0.214,0.228,0.161,0.078,0.319,0.578,0.799,0.420,0.525,0.551,0.405,0.235,0.864,0.002,0.0,0.280,0.500
29968,29969,2022,Jalen Brunson,DAL,4,25.0,SG,79,31.9,6.4,12.8,0.502,1.2,3.2,0.373,5.2,9.6,0.545,0.549,2.3,2.7,0.840,0.5,3.4,3.9,4.8,0.8,0.0,1.6,1.9,16.3,43.0,57.0,NaN,NaN,NaN,4.4,2.3,71.0,29.0,60.0,14.0,83.0,44.0,945.0,26.0,38.0,13.1,0.750,0.147,0.288,0.189,0.126,0.250,0.545,0.703,0.524,0.518,0.449,0.373,0.262,0.809,0.005,5.0,0.298,0.507
30708,30709,2023,Jalen Brunson,NYK,5,26.0,PG,68,35.0,8.6,17.6,0.491,2.0,4.7,0.416,6.7,12.8,0.519,0.547,4.8,5.8,0.829,0.6,3.0,3.5,6.2,0.9,0.2,2.1,2.2,24.0,100.0,NaN,NaN,NaN,NaN,2.4,-1.9,75.0,39.0,70.0,18.0,169.0,77.0,1036.0,50.0,51.0,12.9,0.731,0.149,0.331,0.172,0.079,0.269,0.519,0.612,0.504,0.471,0.511,0.416,0.166,0.575,0.002,2.0,0.205,0.424


In [9]:
pre = merged_player.fillna(0).copy()
player_stats = pre[similarity_columns].dropna(how='all')

# Alternatively, keep rows with any non-NaN values
# player_stats = merged_player[similarity_columns].dropna(how='any')

In [10]:
# Normalize the data
scaler = StandardScaler()
player_stats_scaled = scaler.fit_transform(player_stats)

# Choose a specific row (e.g., the player to compare)
chosen_row_index = 31045  # Adjust as needed
chosen_row = player_stats_scaled[chosen_row_index].reshape(1, -1)

# Calculate cosine similarity between the chosen row and all others
similarities = cosine_similarity(player_stats_scaled, chosen_row).flatten()

In [11]:
short = ['season', 'player', 'experience', 'age', 'tm', 'pos', 'g', 'avg_dist_fga', 'ft_percent',
       'trb_per_game', 'ast_per_game', 'stl_per_game', 'blk_per_game',
       'tov_per_game', 'pts_per_game', 'similarity_score']
shai_comp = merged_player.copy()
shai_comp['similarity_score'] = similarities
shai_sorted = shai_comp.sort_values(by='similarity_score', ascending=False)
best_shai = shai_sorted.drop_duplicates(subset='player', keep='first')
best_shai_f = best_shai[short]
best_shai_f.head(6)

,season,player,experience,age,tm,pos,g,avg_dist_fga,ft_percent,trb_per_game,ast_per_game,stl_per_game,blk_per_game,tov_per_game,pts_per_game,similarity_score
31045,2023,Shai Gilgeous-Alexander,5,24.0,OKC,PG,68,9.8,0.905,4.8,5.5,1.6,1.0,2.8,31.4,1.000000
20749,2007,Kobe Bryant,11,28.0,LAL,SG,77,14.5,0.868,5.7,5.4,1.4,0.5,3.3,31.6,0.972550
20063,2006,Dwyane Wade,3,24.0,MIA,SG,75,9.9,0.783,5.7,6.7,1.9,0.8,3.6,27.2,0.963730
26010,2016,Jimmy Butler,5,26.0,CHI,SG,67,12.0,0.832,5.3,4.8,1.6,0.6,2.0,20.9,0.962310
20311,2006,Paul Pierce,8,28.0,BOS,SF,79,12.6,0.772,6.7,4.7,1.4,0.4,3.5,26.8,0.957635
29829,2022,DeMar DeRozan,13,32.0,CHI,PF,76,13.0,0.877,5.2,4.9,0.9,0.3,2.4,27.9,0.956253


In [12]:
# Choose a specific row (e.g., the player to compare)
chosen_row_index = 30708  # Adjust as needed
chosen_row1 = player_stats_scaled[chosen_row_index].reshape(1, -1)

# Calculate cosine similarity between the chosen row and all others
similarities1 = cosine_similarity(player_stats_scaled, chosen_row1).flatten()

brun_comp = merged_player.copy()
brun_comp['similarity_score'] = similarities1
brun_sorted = brun_comp.sort_values(by='similarity_score', ascending=False)
best_brun = brun_sorted.drop_duplicates(subset='player', keep='first')
best_brun_f = best_brun[short]
best_brun_f.head(6)

,season,player,experience,age,tm,pos,g,avg_dist_fga,ft_percent,trb_per_game,ast_per_game,stl_per_game,blk_per_game,tov_per_game,pts_per_game,similarity_score
30708,2023,Jalen Brunson,5,26.0,NYK,PG,68,12.9,0.829,3.5,6.2,0.9,0.2,2.1,24.0,1.000000
20234,2006,Manu Ginóbili,4,28.0,SAS,SG,65,12.0,0.778,3.5,3.6,1.6,0.4,1.9,15.1,0.944111
29845,2022,Derrick White,5,27.0,TOT,SG,75,15.4,0.864,3.5,4.9,0.9,0.8,1.6,13.2,0.938779
28056,2019,Mike Conley,12,31.0,MEM,PG,70,15.3,0.845,3.4,6.4,1.3,0.3,1.9,21.1,0.934593
30646,2023,Franz Wagner,2,21.0,ORL,SF,80,12.0,0.842,4.1,3.5,1.0,0.2,2.1,18.6,0.928970
31241,2024,Coby White,5,23.0,CHI,PG,79,15.5,0.838,4.5,5.1,0.7,0.2,2.1,19.1,0.928104
